In [1]:
!pip install "autogen-ext[openai]"
!pip install langchain-community
!pip install langchain-core
!pip install -U "autogen-agentchat"


In [2]:
import os


from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

secret_value_0 = user_secrets.get_secret("OPENAI_API_KEY")
secret_value_1 = user_secrets.get_secret("SERPER_API_KEY")

from langchain_core.tools import StructuredTool

os.environ['OPENAI_API_KEY'] = secret_value_0
os.environ['SERPER_API_KEY'] = secret_value_1

from autogen_ext.tools.langchain import LangChainToolAdapter

from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_community.agent_toolkits import FileManagementToolkit
from langchain.tools import tool

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from autogen_ext.models.openai import OpenAIChatCompletionClient

from IPython.display import display, Markdown, HTML
import asyncio
from autogen_agentchat.conditions import MaxMessageTermination
from IPython.display import display, HTML, Markdown
import re

!rm -rf /kaggle/working/sandbox

os.makedirs("sandbox", exist_ok=True)

In [3]:
serper = GoogleSerperAPIWrapper()

def search_func(query: str) -> str:
    """Search the internet for real-time data."""
    return serper.run(query)

search_tool = LangChainToolAdapter(
    StructuredTool.from_function(
        func=search_func,
        name="internet_search",
        description="Search the internet for real-time data"
    )
)

file_tools = [LangChainToolAdapter(t) for t in FileManagementToolkit(root_dir="sandbox").get_tools()]
tools = [search_tool] + file_tools

In [4]:
model_fast = OpenAIChatCompletionClient(model="gpt-4o-mini")
model_strong = OpenAIChatCompletionClient(model="gpt-4o-2024-11-20")

In [5]:
data_collector = AssistantAgent(
    name="DataCollector",
    model_client=model_fast,
    tools=tools,
    system_message="Collect ONLY raw real-time data (prices, news, earnings, on-chain). Save everything to raw_data.md"
)

fundamental_analyst = AssistantAgent(
    name="FundamentalAnalyst",
    model_client=model_strong,
    tools=tools,
    system_message="Deep fundamental analysis only. Save full report to fundamental_analysis.md"
)

technical_analyst = AssistantAgent(
    name="TechnicalAnalyst",
    model_client=model_fast,
    tools=tools,
    system_message="Technical analysis, price action, sentiment, chart descriptions. Save to technical_analysis.md"
)

# portfolio_manager = AssistantAgent(
#     name="PortfolioManager",
#     model_client=model_strong,
#     tools=tools,
#     system_message="""You are the final decision maker.

# CRITICAL: You MUST use the 'write_file' tool to save research_report.md FIRST.

# Steps:
# 1. Read ALL files in sandbox/ using 'list_directory' and 'read_file' tools
# 2. Synthesize everything into one comprehensive analysis
# 3. Write the FULL research_report.md using 'write_file' tool
# 4. ONLY AFTER the file is saved, output your final decision

# Final response format (EXACTLY, after file is written):

# ### Final Investment Decision
# **NVDA** → BUY|HOLD|SELL | [1-sentence reason]
# **BTC**  → BUY|HOLD|SELL | [1-sentence reason] 
# **Confidence** → High|Medium|Low

# Do NOT speak until research_report.md exists."""
# )


portfolio_manager = AssistantAgent(
    name="PortfolioManager",
    model_client=model_strong,
    tools=tools,
    system_message="""You are the final decision maker. You are extremely strict.
    
    NEVER speak unless the FIRST THREE files exist:
    - raw_data.md
    - fundamental_analysis.md
    - technical_analysis.md
   
    
    MANDATORY STEPS (do not skip any):
    1. Use list_directory on "sandbox" → confirm the first 3 files exist
    2. If any of the first 3 files are missing → STOP and say ONLY: "Waiting for raw_data.md, fundamental_analysis.md, and technical_analysis.md to be created."
    3. Once all 3 exist → use read_file on each
    4. Synthesize into one professional research report
    5. Use write_file exactly like this:
       filename="research_report.md"
       content=your full report
    6. ONLY after write_file tool confirms success, output EXACTLY:
    
    Final Investment Decision
    
    **NVDA** → BUY | [8-15 word reason]
    **BTC** → HOLD | [8-15 word reason]
    **Confidence** → High
    
    Example:
    Final Investment Decision
    
    **NVDA** → BUY | Dominant AI revenue growth with $5T+ pipeline confirmed
    **BTC** → HOLD | Bearish momentum but institutional accumulation continues
    **Confidence** → High
    
    NO ###. NO extra text. NO reasoning outside these lines."""
    )

In [6]:
termination = MaxMessageTermination(30)

team = RoundRobinGroupChat(participants=[data_collector, fundamental_analyst, technical_analyst, portfolio_manager],termination_condition=termination)

team.next_speaker = data_collector

result = await team.run(task="Launch complete autonomous investment research on NVDA (NVIDIA stock) and BTC (Bitcoin) using real-time data as of November 2025.")

In [7]:
final_text = result.messages[-1].content

if "Final Investment Decision" in final_text:
    block = final_text.split("Final Investment Decision", 1)[1]
else:
    block = final_text

nvda = re.search(r"\*\*NVDA\*\*.*?→.*?(\bBUY\b|\bHOLD\b|\bSELL\b)", block, re.I)
nvda = nvda.group(1).upper() if nvda else "BUY"

btc = re.search(r"\*\*BTC\*\*.*?→.*?(\bBUY\b|\bHOLD\b|\bSELL\b)", block, re.I)
btc = btc.group(1).upper() if btc else "BUY"

conf = re.search(r"Confidence.*?→.*?(\bHigh\b|\bMedium\b|\bLow\b|\bVery High\b|\bVery Low\b)", block, re.I)
confidence = conf.group(1) if conf else "High"

final = f"**NVDA** → {nvda}\n**BTC**  → {btc}\n**Confidence** → {confidence}"

file_count = len([f for f in os.listdir("sandbox") if f.endswith(".md")])

display(HTML(f"""
<style>
  .header {{ background: linear-gradient(135deg, #0f1620, #1e3c72); padding: 32px; border-radius: 16px; 
            text-align: center; color: white; box-shadow: 0 12px 40px rgba(0,0,0,0.7); margin: 30px 0; }}
  .box {{ background: #0d1117; color: #00ff9d; padding: 45px; border-radius: 16px; 
         border: 2px solid #00ff9d; text-align: center; font-size: 1.9em; line-height: 2.4; 
         box-shadow: 0 0 50px rgba(0,255,157,0.35); margin: 40px auto; max-width: 900px; font-weight: 500; }}
  .signal {{ padding: 20px 60px; border-radius: 70px; font-weight: 900; font-size: 1.8em; 
            display: inline-block; letter-spacing: 2px; }}
  .buy  {{ background: #00ff9d; color: #000; box-shadow: 0 0 40px rgba(0,255,157,0.7); }}
  .hold {{ background: #f0b90b; color: #000; }}
  .sell {{ background: #ff2d55; color: #fff; }}
</style>

<div class="header">
  <h1 style="margin:0; font-size:2.7em; letter-spacing:1px;">Autonomous Alpha Engine</h1>
  <p style="margin:10px 0 0; color:#999; font-size:1.2em;">
    NVDA + BTC • Real-Time Multi-Agent Research • November 2025
    <span style="color:#00ff9d; font-weight:500;"> | Files: {file_count}</span>
  </p>
</div>
"""))

display(Markdown(f"<div class='box'>Final Investment Decision\n\n{final}</div>"))

badge = '<span class="signal sell">SELL</span>' if "SELL" in (nvda + btc) else \
        '<span class="signal hold">HOLD</span>' if "HOLD" in (nvda + btc) else \
        '<span class="signal buy">STRONG BUY</span>'

display(HTML(f'<div style="text-align:center; margin:60px 0;">{badge}</div>'))

print("Available files:")
for f in sorted(os.listdir("sandbox")):
    if f.endswith(".md"):
        print(f"  {f}")

<div class='box'>Final Investment Decision

**NVDA** → BUY
**BTC**  → BUY
**Confidence** → High</div>

Available files:
  final_investment_decision.md
  fundamental_analysis.md
  portfolio_recommendations.md
  raw_data.md
  research_report.md
  technical_analysis.md


In [8]:
with open("sandbox/research_report.md", "r", encoding="utf-8") as f:
    report = f.read()

display(HTML("""
<div style="background: linear-gradient(to bottom, #141e30, #243b55); padding: 30px; border-radius: 15px; color: white; text-align: center;">
  <h1 style="color: #00ff9d; text-shadow: 0 0 15px #00ff9d;">Complete Autonomous Research Report</h1>
</div>
"""))

display(Markdown(report))

**Comprehensive Investment Research Report: NVIDIA (NVDA) and Bitcoin (BTC)**

### Overview from Raw Data
1. **NVIDIA Stock Price Movements:**
   - November 19, 2025: $186.52
   - November 18, 2025: $181.36
   - November 17, 2025: $186.60
   - November 14, 2025: $190.17

2. **Bitcoin Price Trends:**
   - November 19, 2025: $91,757
   - November 18, 2025: $90,000
   - November 16, 2025: $95,508.31

3. **NVIDIA News Highlights:**
   - Record revenue for Q3 2025: $57.0 billion, a 62% YoY increase.
   - Outperformed earnings expectations with $1.30 per share.

4. **Bitcoin Key Observations:**
   - Continued price decline below $90,000, erasing all 2025 gains.
   - Increased whale transactions, testing support near $91,700.

### Fundamental Analysis Summary
**NVIDIA (NVDA):**
- Revenue surged to $57.0 billion in Q3 2025, marking a significant 62% YoY growth.
- Earnings exceeded expectations at $1.30 per share.
- Current Price: $186.52, supported by strong institutional demand and AI-sector growth prospects.

**Bitcoin (BTC):**
- Current Price: $91,757.
- Experienced a notable fall below $90,000 due to increased selling pressure in November 2025.
- On-chain metrics show heightened whale activity and stable support above $91,700.

### Technical Analysis Overview
**NVIDIA (NVDA):**
- Price fluctuations observed between $180-$200.
- Key support levels identified at $179.65 and $181.36.

**Bitcoin (BTC):**
- Testing critical support near $91,700 with resistance forecasted around $95,000.
- On-chain data reflects declining momentum but elevated whale accumulation.

### Investment Recommendations
1. **NVDA (NVIDIA):**
   **Buy**: Strong AI market positioning and recent revenue growth reinforce investor confidence. Prices near support levels make this a favorable entry point.

2. **BTC (Bitcoin):**
   **Hold**: While bearish momentum persists, significant whale accumulation signals potential for recovery above current support levels.

### Conclusion
NVIDIA’s substantial earnings growth and robust AI pipeline deployment provide compelling reasons to invest, while Bitcoin’s price dip amidst whale accumulation warrants cautious optimism and holding strategies.